# 1. Business Understanding

In [ ]:
# Tujuan: Mengklasifikasikan jenis bunga iris berdasarkan karakteristik (panjang/panjang kelopak dan sepal).

# 2. Data Understanding

In [14]:
# Import Library yang dibutuhkan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import davies_bouldin_score
from sklearn.decomposition import PCA


In [2]:
# Memuat dataset Iris yang sudah tersedia di scikit-learn
# Dataset ini berisi data pengukuran bunga iris
iris = datasets.load_iris()

# Mengambil data fitur (X) yang terdiri dari 4 pengukuran:
# - sepal length (cm)
# - sepal width (cm) 
# - petal length (cm)
# - petal width (cm)
X = iris.data

# Mengambil data target/label (y) yang berisi kelas bunga iris:
# - 0: Iris Setosa
# - 1: Iris Versicolor  
# - 2: Iris Virginica
y = iris.target



In [ ]:
# Mengubah data array X menjadi DataFrame pandas untuk memudahkan visualisasi dan analisis
# Parameter X: data fitur bunga iris (sepal length, sepal width, petal length, petal width)
# Parameter columns: nama-nama fitur dari iris.feature_names
df = pd.DataFrame(X, columns=iris.feature_names)

# Menambahkan kolom 'species' yang berisi nama spesies bunga iris
# iris.target_names[y] akan mengambil nama spesies sesuai dengan indeks di array y
# Misal y=0 akan mengambil 'setosa', y=1 akan mengambil 'versicolor', dst
df['species'] = iris.target_names[y]

# Menampilkan 5 baris pertama dari DataFrame untuk melihat struktur data
# Akan menampilkan nilai sepal length, sepal width, petal length, petal width dan species
print(df.head())

In [ ]:
# Menampilkan informasi dataset menggunakan method info()
# Method info() akan menampilkan:
# - Jumlah baris data (entries)
# - Tipe data setiap kolom (dtype) 
# - Jumlah data non-null
# - Penggunaan memori
print("Informasi Dataset:")
print(df.info())

In [ ]:
# Menampilkan statistik deskriptif dari dataset iris menggunakan method describe()
# Method describe() akan menampilkan ringkasan statistik untuk setiap kolom numerik:
# - count: jumlah data
# - mean: rata-rata 
# - std: standar deviasi
# - min: nilai minimum
# - 25%: kuartil pertama (Q1)
# - 50%: median/kuartil kedua (Q2)
# - 75%: kuartil ketiga (Q3)
# - max: nilai maksimum
print("\nStatistik Deskriptif:")
print(df.describe())


In [ ]:
# # Visualisasi korelasi
# plt.figure(figsize=(10, 8))
# sns.heatmap(data.corr(), annot=True, cmap='coolwarm', fmt=".2f")
# plt.title("Heatmap Korelasi Antar Fitur")
# plt.show()

# 3. Data Preparation

In [ ]:
# Melakukan pengecekan nilai yang hilang (missing values) pada dataset
# df.isnull() akan menghasilkan DataFrame boolean dimana True menandakan nilai yang hilang
# df.isnull().sum() akan menghitung jumlah nilai True (missing values) untuk setiap kolom
# Hasil yang diharapkan adalah 0 untuk semua kolom, yang berarti tidak ada missing values
print("\nCek Missing Values:")
print(df.isnull().sum())


In [7]:
# Melakukan normalisasi data menggunakan StandardScaler
# StandardScaler akan mengubah data sehingga memiliki mean=0 dan variance=1
# Hal ini penting untuk algoritma KMeans karena:
# - Mencegah fitur dengan skala besar mendominasi perhitungan jarak
# - Membuat semua fitur memiliki kontribusi yang setara
# - Meningkatkan kinerja dan akurasi model

# Membuat objek StandardScaler
scaler = StandardScaler()

# Melakukan fit dan transform data X sekaligus
# - fit: menghitung mean dan std dari data
# - transform: melakukan normalisasi dengan (x - mean) / std
# Hasil transformasi disimpan ke dalam X_scaled
X_scaled = scaler.fit_transform(X)


In [ ]:
# Memvisualisasikan data yang telah dinormalisasi menggunakan pairplot
# - sns.pairplot() digunakan untuk membuat matriks scatter plot
# - pd.DataFrame() mengubah array X_scaled menjadi DataFrame dengan nama kolom dari iris.feature_names
# - Pairplot akan menampilkan hubungan antara semua pasangan fitur
# - Berguna untuk melihat distribusi dan pola data setelah normalisasi
# - Diagonal plot menunjukkan distribusi masing-masing fitur
# - Plot lainnya menunjukkan hubungan antara dua fitur yang berbeda
sns.pairplot(pd.DataFrame(X_scaled, columns=iris.feature_names))
plt.show()

# 4. Modelling

In [9]:
# Mencari jumlah cluster optimal menggunakan Elbow Method
# - Metode Elbow digunakan untuk menentukan jumlah cluster (k) yang optimal
# - Caranya dengan mencoba berbagai nilai k dan melihat nilai inertia (within-cluster sum of squares)

# Membuat list kosong untuk menyimpan nilai inertia
inertia = []

# Melakukan iterasi untuk nilai k dari 1 sampai 10
for k in range(1, 11):
    # Membuat model KMeans dengan jumlah cluster = k
    # random_state=42 digunakan untuk hasil yang konsisten/reproducible
    kmeans = KMeans(n_clusters=k, random_state=42)
    
    # Melakukan fitting model dengan data yang telah dinormalisasi (X_scaled)
    kmeans.fit(X_scaled)
    
    # Menyimpan nilai inertia ke dalam list
    # inertia_ adalah atribut yang menyimpan jumlah kuadrat jarak setiap titik ke centroid terdekatnya
    inertia.append(kmeans.inertia_)


In [ ]:
# Membuat visualisasi Elbow Method untuk menentukan jumlah cluster optimal
# - plt.plot() digunakan untuk membuat grafik garis
# - range(1,11) membuat sumbu x dari 1-10 yang merepresentasikan jumlah cluster
# - inertia adalah list yang berisi nilai inertia untuk setiap k
# - marker='o' menambahkan titik pada setiap data point
# - plt.title() memberikan judul pada grafik
# - plt.xlabel() memberikan label pada sumbu x
# - plt.ylabel() memberikan label pada sumbu y
# - plt.show() menampilkan grafik
plt.plot(range(1, 11), inertia, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.show()

In [11]:
# Membuat model KMeans dengan parameter:
# - n_clusters=3: menentukan jumlah cluster yang diinginkan (3 cluster karena ada 3 spesies bunga Iris)
# - random_state=42: mengatur seed random untuk hasil yang konsisten setiap kali kode dijalankan
kmeans = KMeans(n_clusters=3, random_state=42)

# Melakukan fitting model sekaligus prediksi cluster untuk setiap data point
# - fit_predict(): metode untuk melatih model dan memprediksi cluster dalam satu langkah
# - X_scaled: data input yang sudah dinormalisasi
# - Hasilnya disimpan dalam y_kmeans yang berisi label cluster (0, 1, atau 2) untuk setiap data
y_kmeans = kmeans.fit_predict(X_scaled)


In [ ]:
# Menambahkan kolom baru bernama 'cluster' ke DataFrame df
# Nilai kolom diisi dengan hasil prediksi cluster dari KMeans (y_kmeans)
# y_kmeans berisi label cluster (0, 1, atau 2) untuk setiap data point
df['cluster'] = y_kmeans

# Membuat visualisasi scatter plot menggunakan seaborn
# Parameter yang digunakan:
# - x: menggunakan fitur pertama dari dataset iris (sepal length)
# - y: menggunakan fitur kedua dari dataset iris (sepal width) 
# - hue='cluster': memberikan warna berbeda untuk setiap cluster
# - palette='viridis': menggunakan color palette viridis
# - data=df: menggunakan DataFrame df sebagai sumber data
sns.scatterplot(x=df[iris.feature_names[0]], y=df[iris.feature_names[1]], hue='cluster', palette='viridis', data=df)

# Menambahkan judul pada plot
plt.title('KMeans Clustering of Iris Dataset')

# Menampilkan plot
plt.show()


# 5. Evaluation

In [ ]:
# Davies-Bouldin Index (DBI) adalah metrik evaluasi untuk mengukur kualitas clustering
# - Semakin kecil nilai DBI, semakin baik hasil clustering
# - DBI mengukur rasio antara jarak intra-cluster (kohesi) dan jarak inter-cluster (separasi)
# - Rentang nilai DBI adalah dari 0 (terbaik) hingga tak terhingga

# Menghitung nilai DBI menggunakan fungsi davies_bouldin_score dari sklearn
# Parameter:
# - X_scaled: data yang telah dinormalisasi
# - y_kmeans: label cluster hasil prediksi KMeans
dbi_score = davies_bouldin_score(X_scaled, y_kmeans)

# Menampilkan nilai DBI dengan 4 angka desimal
# Menggunakan f-string untuk format output yang rapi
print(f'Davies-Bouldin Index: {dbi_score:.4f}')


# 6. Deployment

In [ ]:
# Melakukan reduksi dimensi menggunakan PCA (Principal Component Analysis)
# - Membuat objek PCA dengan parameter n_components=2 untuk mereduksi menjadi 2 dimensi
# - PCA berguna untuk memvisualisasikan data multidimensi ke dalam 2D
pca = PCA(n_components=2)

# Melakukan fit dan transform data yang telah dinormalisasi (X_scaled)
# - fit: menghitung komponen utama dari data
# - transform: mengubah data asli ke dalam bentuk komponen utama
# - Hasilnya disimpan dalam X_pca
X_pca = pca.fit_transform(X_scaled)

# Membuat DataFrame baru untuk hasil PCA
# - Mengkonversi array X_pca menjadi DataFrame dengan kolom 'PCA1' dan 'PCA2'
# - PCA1 adalah komponen utama pertama
# - PCA2 adalah komponen utama kedua
df_pca = pd.DataFrame(X_pca, columns=['PCA1', 'PCA2'])

# Menambahkan kolom cluster dari hasil KMeans sebelumnya
# - Mengambil label cluster dari DataFrame df
# - Menambahkannya sebagai kolom baru di df_pca
df_pca['cluster'] = df['cluster']

# Membuat visualisasi scatter plot dari hasil PCA
# - x: menggunakan PCA1 sebagai sumbu x
# - y: menggunakan PCA2 sebagai sumbu y
# - hue: memberikan warna berbeda untuk setiap cluster
# - palette: menggunakan skema warna viridis
# - data: menggunakan DataFrame df_pca
sns.scatterplot(x='PCA1', y='PCA2', hue='cluster', palette='viridis', data=df_pca)

# Menambahkan judul pada plot
plt.title('KMeans Clustering of Iris Dataset (PCA Projection)')

# Menampilkan plot
plt.show()
